# Data Class Analysis

This file details maps for each group of data class together with quantitative and distribution metrics.

In [1]:
import os
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
%matplotlib inline
# %matplotlib notebook
pd.set_option("float_format", '{:0.2f}'.format)
pd.set_option('display.max_columns', 30)

pvs = 9

datasets_folder = "E:\\DataSets\\100 Hz"

def getDataset(pvs):

    folder = os.path.join(datasets_folder, "PVS " + str(pvs))
    data_left = pd.read_csv(os.path.join(folder, 'dataset_gps_mpu_left.csv'), float_precision="high")
    data_right = pd.read_csv(os.path.join(folder, 'dataset_gps_mpu_right.csv'), float_precision="high")
    data_labels = pd.read_csv(os.path.join(folder, 'dataset_labels.csv'))

    return {
        "data_left": data_left,
        "data_right": data_right,
        "data_labels": data_labels
    }
def mapClasses(pvs, classes, colors):
    
    dataset = getDataset(pvs)
    data = pd.concat([dataset["data_left"], dataset["data_labels"]], axis=1)

    gps = data[['latitude', 'longitude']]
    focolat = (gps['latitude'].min() + gps['latitude'].max()) / 2
    focolon = (gps['longitude'].min() + gps['longitude'].max()) / 2
    maps = folium.Map(location=[focolat, focolon], zoom_start=14)

    grouper = data.groupby(["latitude","longitude"]).mean().round(0)

    for i in range(0, len(classes)):
    
        classe = classes[i]
        color = colors[i]
        points = grouper[grouper[classe] == 1].index.values.reshape(-1)
        
        for point in points:
            folium.Circle(point, color=color, radius=0.1).add_to(maps)

    return maps

def mapClassesSideBySyde(pvs, classes, colors):
    
    map1 = mapClasses(pvs[0], classes, colors)
    map2 = mapClasses(pvs[1], classes, colors)
    map3 = mapClasses(pvs[2], classes, colors)
    
    html_map = HTML("""
    <iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width:33%; margin: 0 auto; border: 1px solid black"></iframe>
    <iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width:33%; margin: 0 auto; border: 1px solid black"></iframe>
    <iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width:33%; margin: 0 auto; border: 1px solid black"></iframe>
    """.format(
        map1.get_root().render().replace('"', '&quot;'),500,500,
        map2.get_root().render().replace('"', '&quot;'),500,500,
        map3.get_root().render().replace('"', '&quot;'),500,500)
    )

    display(html_map)
    
def makeLegend(classes_names, colors):
    
    html_legend = """
    <style>
        .legend { list-style: none; }
        .legend li { float: left; margin-right: 10px; }
        .legend span { border: 1px solid #ccc; float: left; width: 12px; height: 12px; margin: 2px; }
    </style>
    <ul class="legend" style="list-style: none;">
    """
    
    for i in range(0, len(classes_names)):
        name = classes_names[i]
        color = colors[i]
        
        html_legend += """
        <li><span style="background-color: {}"></span> {}</li>
        """.format(color, name)
    
    html_legend += """
    </ul>
    """
    
    display(HTML(html_legend))
    
def metrics(data_class):
    
    list_data = []
    
    for i in range(1,10):
        data = getDataset(i)
        list_data.append(data["data_labels"][data_class].sum())
       
    data = pd.DataFrame(list_data)
    data["Total"] = data.sum(axis=1)
    
    for classe in data_class:
        data[classe + "_%"] = round(data[classe]/data["Total"] *100, 2)
        
    data.index = np.arange(1, len(data) + 1)
    data.index = data.index.rename("PVS")
    return data

## Road Surface Type

In [2]:
makeLegend(["Dirt Road", "Cobblestone Road", "Asphalt Road"], ["red", "green", "blue"])
mapClasses(pvs, ["dirt_road", "cobblestone_road", "asphalt_road"], ["red", "green", "blue"])

Metrics for left and right side

In [3]:
metrics(["dirt_road", "cobblestone_road", "asphalt_road"])

,dirt_road,cobblestone_road,asphalt_road,Total,dirt_road_%,cobblestone_road_%,asphalt_road_%
PVS,,,,,,,
1,25868,61659,56509,144036,17.96,42.81,39.23
2,44618,20737,59330,124685,35.78,16.63,47.58
3,28659,26143,51014,105816,27.08,24.71,48.21
4,23903,57670,50919,132492,18.04,43.53,38.43
5,60539,18143,55195,133877,45.22,13.55,41.23
6,23888,31641,40750,96279,24.81,32.86,42.32
7,23778,54224,50546,128548,18.50,42.18,39.32
8,44939,18825,59854,123618,36.35,15.23,48.42
9,23153,25182,43220,91555,25.29,27.50,47.21


## Road Surface Condition

In [4]:
makeLegend(["Paved Road", "Unpaved Road"], ["blue", "red"])
mapClasses(pvs, ["paved_road", "unpaved_road"], [ "blue", "red"])

Metrics for left and right side

In [5]:
metrics(["paved_road", "unpaved_road"])

,paved_road,unpaved_road,Total,paved_road_%,unpaved_road_%
PVS,,,,,
1,118168,25868,144036,82.04,17.96
2,80066,44618,124684,64.22,35.78
3,77157,28659,105816,72.92,27.08
4,108589,23903,132492,81.96,18.04
5,73338,60539,133877,54.78,45.22
6,72391,23888,96279,75.19,24.81
7,104770,23778,128548,81.50,18.50
8,78679,44939,123618,63.65,36.35
9,68402,23153,91555,74.71,25.29


## Speed Bump

In [6]:
makeLegend(["Speed Bump Asphalt", "Speed Bump Cobblestone"], ["blue", "red"])
mapClasses(pvs, ["speed_bump_asphalt", "speed_bump_cobblestone"], [ "blue", "red"])

Metrics for left and right side

In [7]:
metrics(["speed_bump_asphalt", "speed_bump_cobblestone", "no_speed_bump"])

,speed_bump_asphalt,speed_bump_cobblestone,no_speed_bump,Total,speed_bump_asphalt_%,speed_bump_cobblestone_%,no_speed_bump_%
PVS,,,,,,,
1,626,2829,140581,144036,0.43,1.96,97.60
2,2006,0,122679,124685,1.61,0.00,98.39
3,299,310,105207,105816,0.28,0.29,99.42
4,530,2604,129358,132492,0.40,1.97,97.63
5,1943,0,131934,133877,1.45,0.00,98.55
6,281,325,95673,96279,0.29,0.34,99.37
7,568,2308,125672,128548,0.44,1.80,97.76
8,1837,0,121781,123618,1.49,0.00,98.51
9,308,335,90914,91557,0.34,0.37,99.30


## Road Roughness Condition

In [8]:
makeLegend(["Good Road", "Regular Road", "Bad Road"], ["green", "yellow", "red"])
mapClasses(pvs, ["good_road_left", "regular_road_left", "bad_road_left"], ["green", "yellow", "red"])

Metrics for left side

In [9]:
metrics(["good_road_left", "regular_road_left", "bad_road_left"])

,good_road_left,regular_road_left,bad_road_left,Total,good_road_left_%,regular_road_left_%,bad_road_left_%
PVS,,,,,,,
1,56577,65304,22155,144036,39.28,45.34,15.38
2,56086,37921,30677,124684,44.98,30.41,24.60
3,50923,46638,8255,105816,48.12,44.07,7.80
4,50744,62838,18910,132492,38.30,47.43,14.27
5,53715,32999,47163,133877,40.12,24.65,35.23
6,40446,51051,4782,96279,42.01,53.02,4.97
7,49855,64743,13950,128548,38.78,50.36,10.85
8,56466,29949,37203,123618,45.68,24.23,30.10
9,42557,38919,10079,91555,46.48,42.51,11.01


Metrics for right side

In [10]:
metrics(["good_road_right", "regular_road_right", "bad_road_right"])

,good_road_right,regular_road_right,bad_road_right,Total,good_road_right_%,regular_road_right_%,bad_road_right_%
PVS,,,,,,,
1,56595,66038,21403,144036,39.29,45.85,14.86
2,55926,38057,30701,124684,44.85,30.52,24.62
3,50906,47384,7526,105816,48.11,44.78,7.11
4,50732,64102,17658,132492,38.29,48.38,13.33
5,53510,32587,47780,133877,39.97,24.34,35.69
6,40453,50180,5646,96279,42.02,52.12,5.86
7,50040,67230,11278,128548,38.93,52.30,8.77
8,56617,32301,34700,123618,45.80,26.13,28.07
9,42711,42625,6219,91555,46.65,46.56,6.79
